# Research That Involves Scores From All Games

In [17]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
import datetime

### Investigate Home/Away Dispairity

In [2]:
url = 'https://www.pro-football-reference.com/boxscores/game-scores.htm'

# Get page content
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

### Get Links For All Games Per Score

In [3]:
table = soup.find('table', attrs={'class': 'sortable stats_table'})
trs = table.find_all('tr')  # each tr contains a link to each score

# Use trs that have data
trs = trs[1:]  # trs[0] contains only header info

# Get all links
links = []
prefix = 'https://www.pro-football-reference.com'
for td in trs:
    hrefs = [link.get('href') for link in td.find_all('a')]
    links.append(prefix + str(hrefs[0]))

### Get All Game Data For Each Score

In [10]:
def get_games_per_score(url):
    # Get page content
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Find the table
    table = soup.find('table', attrs={'id': 'games'})  

    # Get the games data
    games = {}
    for i, row in enumerate(table.find_all('tr')[1:]):
        row_dict = {}  # re-initialize row_dict
        for col in row.find_all('td'):
            # Set row dictionary values
            col_name = col['data-stat']

            # Determine special columns
            if col_name=='winner':
                # winner_id
                try:
                    site = col.find('a').get('href')
                    row_dict['winner_id'] = re.search(r'/teams/([A-Za-z0-9]+)/', site).group(1)
                except:
                    row_dict['winner_id'] = None
            elif col_name=='loser':
                # loser_id
                try:
                    site = col.find('a').get('href')
                    row_dict['loser_id'] = re.search(r'/teams/([A-Za-z0-9]+)/', site).group(1)
                except:
                    row_dict['loser_id'] = None
            elif col_name=='game_location':
                # home_win
                row_dict['home_win'] = 0 if r'@' in col.text else 1
                continue
            elif col_name=='boxscore_word':
                # box_score
                try:
                    site = col.find('a').get('href')
                    row_dict['boxscore'] = 'https://www.pro-football-reference.com'+site
                except:
                    row_dict['boxscore'] = None
                continue
            elif col_name=='game_outcome':
                continue

            row_dict[col_name] = col.text

        # Add to games dictionary
        games[i] = row_dict

    return games


### Loop Thru and Combine All Scores

In [11]:
# Look thru game score links
# games = {}
# for i, link in enumerate(links):
#     # Print Status Update
#     win, loss = re.search(r'pts_win=(\d+)\&pts_lose=(\d+)', link).groups()
#     print(f'Retrieving {win}-{loss} Score....{i+1} of {len(links)}')

#     # Dictionary for each score
#     new_games_raw = get_games_per_score(link)

#     # Rename keys so that no dupes exist
#     key_prefix = f'{i:04}'
#     new_games = {(key_prefix+str(k)): new_games_raw[k] 
#                     for k in new_games_raw.keys()}

#     # Combine old games and new games
#     games = {**games, **new_games}
#     time.sleep(2)

# df = pd.DataFrame.from_dict(games, orient='index')

Retrieving 7-7 Score....1 of 1072
Retrieving 7-3 Score....2 of 1072
Retrieving 30-21 Score....3 of 1072
Retrieving 28-23 Score....4 of 1072
Retrieving 37-21 Score....5 of 1072
Retrieving 42-7 Score....6 of 1072
Retrieving 26-0 Score....7 of 1072
Retrieving 21-19 Score....8 of 1072
Retrieving 42-14 Score....9 of 1072
Retrieving 37-31 Score....10 of 1072
Retrieving 33-27 Score....11 of 1072
Retrieving 16-3 Score....12 of 1072
Retrieving 17-12 Score....13 of 1072
Retrieving 41-21 Score....14 of 1072
Retrieving 14-9 Score....15 of 1072
Retrieving 19-10 Score....16 of 1072
Retrieving 33-17 Score....17 of 1072
Retrieving 45-21 Score....18 of 1072
Retrieving 35-24 Score....19 of 1072
Retrieving 45-17 Score....20 of 1072
Retrieving 20-12 Score....21 of 1072
Retrieving 28-3 Score....22 of 1072
Retrieving 45-7 Score....23 of 1072
Retrieving 37-10 Score....24 of 1072
Retrieving 35-0 Score....25 of 1072
Retrieving 31-6 Score....26 of 1072
Retrieving 33-30 Score....27 of 1072
Retrieving 26-16 Score

In [21]:
# Write to csv
today = datetime.datetime.now()
df.to_csv(f'data/all_nfl_games{today.year}_{today.month}_{today.day}.csv')

In [22]:
import get_all_game_data
get_all_game_data.get_data()

KeyboardInterrupt: 